# INFERENCE

Predict function of the np models 
with pre computed context encodings i forgot the batch size i was using 
to measure the memory change i used either gpustat or nvidia smi, making sure to clean the memory after every inference 

## Example of Evaluation Run with Defined Num Context and Target

In [1]:
%run gp_track.py --mode eval --expid lbanp-num_latents-64_rbf --model lbanp --num_latents 64 --custom_eval True --num_tar 10 --num_ctx 10 --eval_num_batches 1

Overriding argument num_latents: 64
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
True
generating evaluation sets...
Generating Evaluation Sets with rbf kernel
Generating Evaluation sets with context and target points:   10 10


100%|##########| 1/1 [00:00<00:00,  9.08it/s]


Starting evaluation


  0%|          | 0/1 [00:00<?, ?it/s]

Output from nvidia-smi: ['memory.used [MiB]\r', '218']
SMI1: Initial GPU memory usage: [218]


100%|##########| 1/1 [00:00<00:00,  1.19it/s]
lbanp:lbanp-num_latents-64_rbf rbf tar_ll 0.8357 loss -0.8357 (0.847 secs)


Memory used for inference: 0.00244140625 MB
Output from nvidia-smi: ['memory.used [MiB]\r', '222']
SMI1: Final GPU memory usage: [222]
SMI1: GPU memory usage during execution: [4]
Eval has finished


## MEMORY AND TIME COMPLEXITY MEASUREMENT
This needs to pre compute the encodings

In [1]:
import os
directory = "/rds/user/fz287/hpc-work/MLMI4/results/gp/lbanp"
# List files in the directory
files = os.listdir(directory)
# Print the list of files
for file in files:
    print(file)

lbanp-num_latents-128
lbanp-num_latents-128_matern
lbanp-num_latents-128_periodic
lbanp-num_latents-128_rbf
lbanp-num_latents-64_rbf
lbanp-num_latents-8
lbanp-num_latents-8_matern
lbanp-num_latents-8_periodic
lbanp-wamup
lbanp1-num_latents-8
lbanp_warmup


In [171]:
directory = "/rds/user/fz287/hpc-work/MLMI4/results/gp/isanp2"
# List files in the directory
files = os.listdir(directory)
# Print the list of files
for file in files:
    print(file)

isanp2-num_latents-128_matern_0
isanp2-num_latents-128_periodic_0
isanp2-num_latents-128_rbf_0
isanp2-num_latents-128_rbf_100
isanp2-num_latents-16_rbf_100
isanp2-num_latents-256_rbf_100
isanp2-num_latents-32_rbf_100
isanp2-num_latents-64_rbf_100
isanp2-num_latents-8_matern_0
isanp2-num_latents-8_periodic_0
isanp2-num_latents-8_rbf_0
isanp2-num_latents-8_rbf_100


In [1]:
import os
import torch
from utils.misc import load_module
import yaml

def load_saved_model(model_name, model_id,lbanp_latents=None):
  # Define the directory path
    if lbanp_latents==None:
        directory = "/rds/user/fz287/hpc-work/MLMI4/results/gp/"+model_name+"/"+model_id
    else:
        directory = "/rds/user/fz287/hpc-work/MLMI4/results/gp/"+model_name+"/"+model_id
    
    args={
    'model':model_name,
    # LBANP Arguments
    'num_latents':lbanp_latents,
    'num_latents_per_layer':8,
    'd_model':64,
    'emb_depth':4,
    'dim_feedforward':128,
    'nhead':4,
    'dropout':0.0,
    'num_layers':6,
    # OOD settings
    'eval_kernel':'rbf',
    't_noise':None,
    'root':directory,
    'eval_logfile':None,
    'eval_seed':0,
    'eval_num_batches':1,
    'eval_batch_size':16,
    'max_num_points':50
    }
    
    model_cls = getattr(load_module(f'models/'+args['model']+'.py'), args['model'].upper())

    with open(f'configs/gp/'+args['model']+'.yaml', 'r') as f:
        config = yaml.safe_load(f)

    print(config)

    for key, val in args.items(): # Override the default arguments
        if key in config:
            config[key] = val
            print(f"Overriding argument {key}: {config[key]}")
    
    model = model_cls(**config)
    model.cuda()
    

    # Define the path to the checkpoint file
    checkpoint_path = directory+'/ckpt.tar'
    # Load the checkpoint
    checkpoint = torch.load(checkpoint_path)
    # Access the model state from the loaded checkpoint
    model_state = checkpoint['model']
    # Assuming you have already created your model object, load the model state
    model.load_state_dict(model_state)
    return model, args

In [2]:
from utils.paths import results_path, evalsets_path
import os.path as osp
from utils.log import get_logger, RunningAverage
from data.gp import *
from attrdict import AttrDict
from tqdm import tqdm

In [3]:
evalsets_path

'/rds/user/fz287/hpc-work/MLMI4/evalsets'

In [4]:
# List files in the directory
files = os.listdir(osp.join(evalsets_path, 'gp'))
# Print the list of files
for file in files:
    print(file)

matern-seed0.tar
matern-seed100.tar
periodic-seed0.tar
periodic-seed100.tar
rbf-seed0.tar
rbf-seed100.tar
rbf-seed500.tar


In [4]:
def get_eval_path(args):
    path = osp.join(evalsets_path, 'gp')
    filename = f'rbf-seed500.tar'
    return path, filename

In [5]:
def gen_evalset(args):
    kernel = RBFKernel()
    print(f"Generating Evaluation Sets with rbf kernel")
    print('Generating Evaluation sets with context and target points:  ', args['num_ctx'], args['num_tar'])
    
    sampler = GPSampler(kernel, t_noise=args['t_noise'], seed=args['eval_seed'], num_ctx=args['num_ctx'], num_tar=args['num_tar'])
    batches = []
    for i in tqdm(range(args['eval_num_batches']), ascii=True):
        batches.append(sampler.sample(
            batch_size=args['eval_batch_size'],
            max_num_points=args['max_num_points'],
            device='cuda'))

    #torch.manual_seed(time.time())
    #torch.cuda.manual_seed(time.time())

    path, filename = get_eval_path(args)
    if not osp.isdir(path):
        os.makedirs(path)
    torch.save(batches, osp.join(path, filename))

In [6]:
def create_eval_batches(args, num_ctx, num_tar):
    
    args['num_ctx']=num_ctx
    args['num_tar']=num_tar

    if args['eval_logfile'] is None:
        eval_logfile = f'eval_rbf'
        eval_logfile += '.log'
    else:
        eval_logfile = args['eval_logfile']
        
    filename = os.path.join(args['root'], eval_logfile)
    logger = get_logger(filename, mode='w')
    
    path, filename = get_eval_path(args)
    
    gen_evalset(args)
    
    eval_batches = torch.load(osp.join(path, filename))
    
    #just works with 1 batch
    for batch in tqdm(eval_batches, ascii=True):    
        for key, val in batch.items():
            batch[key] = val.cuda()
    
    return batch #eval_batches

# Function to clear memory cache
def empty_cache():
    gc.collect()
    torch.cuda.empty_cache()

In [7]:
import subprocess
import gc
def get_gpu_memory_usage():
    result = subprocess.run(['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits'],
                            stdout=subprocess.PIPE)
    output = result.stdout.decode('utf-8').strip().split('\n')
    #print("Output from nvidia-smi:", output)

    # Check if the output contains at least two elements
    if len(output) >= 2:
        # Extract memory usage value from the second element
        memory_usage = int(output[1])
        return [memory_usage]
    else:
        # If output format is unexpected, return an empty list
        print("Unexpected output format from nvidia-smi")
        return []
    
# Function to clear memory cache
def empty_cache():
    gc.collect()
    torch.cuda.empty_cache()

In [8]:
torch.manual_seed(0)
torch.cuda.manual_seed(0)

ravg = RunningAverage()

LBANP(
  (embedder): Sequential(
    (0): Linear(in_features=2, out_features=64, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): ReLU(inplace=True)
    (6): Linear(in_features=64, out_features=64, bias=True)
  )
  (encoder): LBANPEncoder(
    (layers): ModuleList(
      (0-5): 6 x LBANPEncoderLayer(
        (latent_self_attn): PreNorm(
          (fn): Attention(
            (to_q): Linear(in_features=64, out_features=64, bias=False)
            (to_kv): Linear(in_features=64, out_features=128, bias=False)
            (dropout): Dropout(p=0.0, inplace=False)
            (to_out): Linear(in_features=64, out_features=64, bias=True)
          )
          (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        )
        (latent_ff): PreNorm(
          (fn): FeedForward(
            (net): Sequential(
              (0): Linear(in_fea

In [9]:
def run_meas(model, eval_batches, maxit=100):
    #precompute encodings
    if model.__class__.__name__!='TNPD':
        encod=model.get_context_encoding(eval_batches)
    
    cm=0 #to save memory metrics
    tm=0 #to save time metrics
    
    for i in range(maxit):
        #start measurement
        empty_cache()
        starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)        
        starter.record()
        initial_memory_usage = get_gpu_memory_usage()
        #print("SMI1: Initial GPU memory usage:", initial_memory_usage)

        #predict with precomputed encodings
        if model.__class__.__name__!='TNPD':
            pred_tar = model.predict(eval_batches['xc'], eval_batches['yc'], eval_batches['xt'], context_encodings=encod)
        else:
            pred_tar = model.predict(eval_batches['xc'], eval_batches['yc'], eval_batches['xt'])

        #check memory
        final_memory_usage = get_gpu_memory_usage()
        #print("SMI1: Final GPU memory usage:", final_memory_usage)
        memory_usage_during_execution = [final - initial for final, initial in zip(final_memory_usage, initial_memory_usage)]
        #print("SMI1: GPU memory usage during execution:", memory_usage_during_execution)
        #check time
        ender.record()
        torch.cuda.synchronize()
        curr_time = starter.elapsed_time(ender)
        #sum memory
        cm+=memory_usage_during_execution[0]
        #sum time
        tm+=curr_time
        del pred_tar
        
    if model.__class__.__name__!='TNPD':
        del encod
        
    return cm/maxit, tm/maxit

In [157]:
#run_meas(model=modelito128rbf, eval_batches=eval_batches10)

AVG MEMORY USED:  268.875


In [158]:
#run_meas(model=modelito128rbf, eval_batches=eval_batches10)

AVG MEMORY USED:  268.625


In [159]:
#run_meas(model=modelito64rbf, eval_batches=eval_batches10)

AVG MEMORY USED:  227.95


In [161]:
#run_meas(model=modelito64rbf, eval_batches=eval_batches100)

AVG MEMORY USED:  226.6


In [12]:
lbanp_8, args64=load_saved_model(model_name='lbanp', model_id='lbanp1-num_latents-8', lbanp_latents=8)


{'num_latents': 8, 'dim_x': 1, 'dim_y': 1, 'd_model': 64, 'emb_depth': 4, 'dim_feedforward': 128, 'nhead': 4, 'dropout': 0.0, 'num_layers': 6, 'norm_first': True}
Overriding argument num_latents: 8
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6


In [10]:
#load models 
lbanp_8, args64=load_saved_model(model_name='lbanp', model_id='lbanp1-num_latents-8', lbanp_latents=8)
lbanp_64, _=load_saved_model(model_name='lbanp', model_id='lbanp-num_latents-64_rbf', lbanp_latents=64)
lbanp_128, _=load_saved_model(model_name='lbanp', model_id='lbanp-num_latents-128_rbf', lbanp_latents=128)
isanp_8, _=load_saved_model(model_name='isanp', model_id='isanp-num_latents-8_rbf', lbanp_latents=8)
isanp_64, _=load_saved_model(model_name='isanp', model_id='isanp-num_latents-64_rbf_0', lbanp_latents=64)
isanp_128, _=load_saved_model(model_name='isanp', model_id='isanp-num_latents-128_rbf_0', lbanp_latents=128)
isanp2_8, _=load_saved_model(model_name='isanp2', model_id='isanp2-num_latents-8_rbf_0', lbanp_latents=8)
isanp2_64, _=load_saved_model(model_name='isanp2', model_id='isanp2-num_latents-64_rbf_100', lbanp_latents=64)
isanp2_128, _=load_saved_model(model_name='isanp2', model_id='isanp2-num_latents-128_rbf_0', lbanp_latents=128)

tnp_d, _=load_saved_model(model_name='tnpd', model_id='tnpd')

{'num_latents': 8, 'dim_x': 1, 'dim_y': 1, 'd_model': 64, 'emb_depth': 4, 'dim_feedforward': 128, 'nhead': 4, 'dropout': 0.0, 'num_layers': 6, 'norm_first': True}
Overriding argument num_latents: 8
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
{'num_latents': 8, 'dim_x': 1, 'dim_y': 1, 'd_model': 64, 'emb_depth': 4, 'dim_feedforward': 128, 'nhead': 4, 'dropout': 0.0, 'num_layers': 6, 'norm_first': True}
Overriding argument num_latents: 64
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
{'num_latents': 8, 'dim_x': 1, 'dim_y': 1, 'd_model': 64, 'emb_depth': 4, 'dim_feedforward': 128, 'nhead': 4, 'dropout': 0.0, 'num_layers': 6, 'norm_first': True}
Overriding argument num_latents: 128

In [10]:
#tnp_d, _=load_saved_model(model_name='tnpd', model_id='tnpd')

{'dim_x': 1, 'dim_y': 1, 'd_model': 64, 'emb_depth': 4, 'dim_feedforward': 128, 'nhead': 4, 'dropout': 0.0, 'num_layers': 6}
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6


In [11]:
#create eval batches, it doesnt matter which args we use for this
eval_batches10= create_eval_batches(args=args64, num_ctx=10, num_tar=1000)
eval_batches150= create_eval_batches(args=args64, num_ctx=150, num_tar=1000)
eval_batches500= create_eval_batches(args=args64, num_ctx=500, num_tar=1000)
eval_batches1000= create_eval_batches(args=args64, num_ctx=1000, num_tar=1000)
eval_batches1500= create_eval_batches(args=args64, num_ctx=1500, num_tar=1000)
eval_batches2000= create_eval_batches(args=args64, num_ctx=2000, num_tar=1000)
eval_batches2500= create_eval_batches(args=args64, num_ctx=2500, num_tar=1000)
eval_batches3000= create_eval_batches(args=args64, num_ctx=3000, num_tar=1000)

#eval_batches1000= create_eval_batches(args=args64, num_ctx=10000, num_tar=1000)

Generating Evaluation Sets with rbf kernel
Generating Evaluation sets with context and target points:   10 1000


100%|############################################################################################| 1/1 [00:00<?, ?it/s]


Generating Evaluation Sets with rbf kernel
Generating Evaluation sets with context and target points:   150 1000


100%|############################################################################################| 1/1 [00:00<?, ?it/s]


Generating Evaluation Sets with rbf kernel
Generating Evaluation sets with context and target points:   500 1000


...

100%|############################################################################################| 1/1 [00:00<?, ?it/s]


Generating Evaluation Sets with rbf kernel
Generating Evaluation sets with context and target points:   2500 1000


100%|############################################################################################| 1/1 [00:00<?, ?it/s]


Generating Evaluation Sets with rbf kernel
Generating Evaluation sets with context and target points:   3000 1000


100%|############################################################################################| 1/1 [00:00<?, ?it/s]


In [ ]:
#modelito64rbf.eval()
#modelito128rbf.eval()

In [18]:
def run_tests(models, models_names,evalbatches, batch_size):
    # Create an empty DataFrame to store results
    results_list = []

    # Loop through each combination of model and evaluation batch
    for i in range(len(models)):
        print(models_names[i])
        for j in range(len(evalbatches)):
            memory, time=run_meas(model=models[i], eval_batches=evalbatches[j], maxit=120)
            results_list.append({'Model': models_names[i], 'Context Size': str(batch_size[j]),
                                  'Memory Used (MB)': memory, 'Inference Time (ms)':time})

    results = pd.DataFrame(results_list)
    return results

In [13]:
def run_tests(models, models_names,evalbatches, batch_size):
    # Create an empty DataFrame to store results
    results_list = []

    # Loop through each combination of model and evaluation batch

    print(models_names)
    for j in range(len(evalbatches)):
        memory, time=run_meas(model=models, eval_batches=evalbatches[j], maxit=120)
        results_list.append({'Model': models_names, 'Context Size': str(batch_size[j]),
                              'Memory Used (MB)': memory, 'Inference Time (ms)':time})

    results = pd.DataFrame(results_list)
    return results

In [14]:
import pandas as pd

# Define the models, model names, and evaluation batches
models=[lbanp_8, lbanp_64, lbanp_128, isanp_8, isanp_64, isanp_128, isanp2_8, isanp2_64, isanp2_128, tnp_d]
models_names=['LBANP(8)', 'LBANP(64)', 'LBANP(128)', 'ISANP(8)', 'ISANP(64)', 'ISANP(128)', 'ISANPv2(8)', 'ISANPv2(64)', 'ISANPv2(128)', 'TNPD']


In [15]:
evalbatches = [eval_batches10, eval_batches150, eval_batches500, eval_batches1000, eval_batches1500, eval_batches2000, eval_batches2500, eval_batches3000]
batch_size = [10, 150, 500, 1000, 1500, 2000, 2500, 3000]

In [21]:
torch.cuda.reset_max_memory_allocated()
empty_cache()
df_lbanp0=run_tests(models=models[0], models_names=models_names[0],evalbatches=evalbatches, batch_size=batch_size)
df_lbanp0.to_csv('lbanp_0inference_results.csv', index=False)

LBANP(8)


In [22]:
torch.cuda.reset_max_memory_allocated()
empty_cache()
df_lbanp1=run_tests(models=models[1], models_names=models_names[1],evalbatches=evalbatches, batch_size=batch_size)
df_lbanp1.to_csv('lbanp_1inference_results.csv', index=False)

C:\Users\jmlr9\AppData\Roaming\Python\Python311\site-packages\torch\cuda\memory.py:303: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(


LBANP(64)


In [23]:
torch.cuda.reset_max_memory_allocated()
empty_cache()
df_lbanp2=run_tests(models=models[2], models_names=models_names[2],evalbatches=evalbatches, batch_size=batch_size)
df_lbanp2.to_csv('lbanp_2inference_results.csv', index=False)

LBANP(128)


In [24]:
torch.cuda.reset_max_memory_allocated()
empty_cache()
df3_isanp=run_tests(models=models[3], models_names=models_names[3],evalbatches=evalbatches, batch_size=batch_size)
df3_isanp.to_csv('isanp_3inference_results.csv', index=False)

ISANP(8)


In [25]:
torch.cuda.reset_max_memory_allocated()
empty_cache()
df4_isanp=run_tests(models=models[4], models_names=models_names[4],evalbatches=evalbatches, batch_size=batch_size)
df4_isanp.to_csv('isanp_4inference_results.csv', index=False)

ISANP(64)


In [26]:
torch.cuda.reset_max_memory_allocated()
empty_cache()
df5_isanp=run_tests(models=models[5], models_names=models_names[5],evalbatches=evalbatches, batch_size=batch_size)
df5_isanp.to_csv('isanp_5inference_results.csv', index=False)

ISANP(128)


In [27]:
torch.cuda.reset_max_memory_allocated()
empty_cache()
df6_isanp2=run_tests(models=models[6], models_names=models_names[6],evalbatches=evalbatches, batch_size=batch_size)
df6_isanp2.to_csv('isanp2_6inference_results.csv', index=False)

ISANPv2(8)


In [28]:
torch.cuda.reset_max_memory_allocated()
empty_cache()
df7_isanp2=run_tests(models=models[7], models_names=models_names[7],evalbatches=evalbatches, batch_size=batch_size)
df7_isanp2.to_csv('isanp2_7inference_results.csv', index=False)

ISANPv2(64)


In [29]:
torch.cuda.reset_max_memory_allocated()
empty_cache()
df8_isanp2=run_tests(models=models[8], models_names=models_names[8],evalbatches=evalbatches, batch_size=batch_size)
df8_isanp2.to_csv('isanp2_8inference_results.csv', index=False)

ISANPv2(128)


In [18]:
import pandas as pd

In [19]:
torch.cuda.reset_max_memory_allocated()
empty_cache()
df_tnpd=run_tests(models=tnp_d, models_names='TNPD',evalbatches=evalbatches[:3], batch_size=batch_size[:3])
df_tnpd.to_csv('tnpd1_inference_results.csv', index=False)

C:\Users\jmlr9\AppData\Roaming\Python\Python311\site-packages\torch\cuda\memory.py:303: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(


TNPD


In [20]:
torch.cuda.reset_max_memory_allocated()
empty_cache()
df_tnpd=run_tests(models=tnp_d, models_names='TNPD',evalbatches=evalbatches[3:5], batch_size=batch_size[3:5])
df_tnpd.to_csv('tnpd2_inference_results.csv', index=False)

TNPD


The following were memory intensive:

torch.cuda.reset_max_memory_allocated()
empty_cache()
df_tnpd=run_tests(models=tnp_d, models_names='TNPD',evalbatches=evalbatches[5], batch_size=batch_size[5:7])
df_tnpd.to_csv('tnpd3_inference_results.csv', index=False)

torch.cuda.reset_max_memory_allocated()
empty_cache()
df_tnpd=run_tests(models=tnp_d, models_names='TNPD',evalbatches=evalbatches[6], batch_size=batch_size[6])
df_tnpd.to_csv('tnpd4_inference_results.csv', index=False)

torch.cuda.reset_max_memory_allocated()
empty_cache()
df_tnpd=run_tests(models=tnp_d, models_names='TNPD',evalbatches=evalbatches[7], batch_size=batch_size[7:9])
df_tnpd.to_csv('tnpd5_inference_results.csv', index=False)

torch.cuda.reset_max_memory_allocated()
empty_cache()
df_tnpd=run_tests(models=tnp_d, models_names='TNPD',evalbatches=evalbatches[8], batch_size=batch_size[8])
df_tnpd.to_csv('tnpd6_inference_results.csv', index=False)

In [35]:
batch_size

[10, 150, 500, 1000, 1500, 2000, 2500, 3000]

In [16]:
models_names[6:9]

['ISANPv2(8)', 'ISANPv2(64)', 'ISANPv2(128)']

In [19]:
torch.cuda.reset_max_memory_allocated()
empty_cache()
df6_isanp2=run_tests(models=models[6:9], models_names=models_names[6:9],evalbatches=evalbatches, batch_size=batch_size)
df6_isanp2.to_csv('isanp2_6inference_results2.csv', index=False)

ISANPv2(8)
ISANPv2(64)
ISANPv2(128)


## INFERENCE RUNS WITH DIFFERENT EVALUATION SEEDS

In [22]:
import os
directory = "/rds/user/fz287/hpc-work/MLMI4/results/gp/isanp"
# List files in the directory
files = os.listdir(directory)
# Print the list of files
for file in files:
    print(file)

isanp-num_latents-128_matern
isanp-num_latents-128_rbf_0
isanp-num_latents-16_rbf_0
isanp-num_latents-16_rbf_100
isanp-num_latents-256_rbf_0
isanp-num_latents-256_rbf_100
isanp-num_latents-32_rbf_0
isanp-num_latents-32_rbf_100
isanp-num_latents-64_rbf_0
isanp-num_latents-64_rbf_100
isanp-num_latents-8_matern
isanp-num_latents-8_periodic
isanp-num_latents-8_rbf
lbanp-num_latents-128_matern
lbanp-num_latents-128_periodic
lbanp-num_latents-8_matern


### RBF

In [20]:
model_names=['cnp','canp','np','anp','bnp','tnpd','tnpa']
modelitos=['cnp','canp','np','anp','bnp','tnpd','tnpa']

for model_name, modelito in zip(model_names, modelitos):
    print('###############MODEL: ',model_name)
    for seed in [0,58,100,250,550,1200]:
        print('###############SEED: ',seed)
        %run gp_track.py --mode eval --expid {model_name} --model {modelito} --custom_eval True --eval_num_batches 3000 --eval_seed {seed}

###############MODEL:  cnp
###############SEED:  0
True
generating evaluation sets...
Generating Evaluation Sets with rbf kernel
Generating Evaluation sets with context and target points:   None None


100%|##########| 3000/3000 [00:12<00:00, 236.38it/s]


Starting evaluation
ROOT:  /rds/user/fz287/hpc-work/MLMI4/results\gp\cnp\cnp


100%|##########| 3000/3000 [00:14<00:00, 201.73it/s]
cnp:cnp rbf ctx_ll 0.5416 tar_ll 0.1377 (14.874 secs)


Eval has finished
###############SEED:  58
True
generating evaluation sets...
Generating Evaluation Sets with rbf kernel
Generating Evaluation sets with context and target points:   None None


...

Eval has finished
###############SEED:  1200
Overriding argument pretrain: False
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
True
generating evaluation sets...
Generating Evaluation Sets with rbf kernel
Generating Evaluation sets with context and target points:   None None


100%|##########| 3000/3000 [00:10<00:00, 299.10it/s]


Starting evaluation
ROOT:  /rds/user/fz287/hpc-work/MLMI4/results\gp\tnpa\tnpa


100%|##########| 3000/3000 [00:25<00:00, 119.68it/s]
tnpa:tnpa rbf tar_ll 1.5008 loss -1.5008 (25.070 secs)


Eval has finished


In [7]:
model_name='lbanp-num_latents-64_rbf'
modelito='lbanp'
latents=64

for seed in [0,58,100,250,550,1200]:
    print('###############SEED: ',seed)
    %run gp_track.py --mode eval --expid {model_name} --model {modelito} --num_latents {latents} --custom_eval True --eval_num_batches 3000 --eval_seed {seed}

Overriding argument num_latents: 64
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
True
generating evaluation sets...
Generating Evaluation Sets with rbf kernel
Generating Evaluation sets with context and target points:   None None


100%|##########| 3000/3000 [00:12<00:00, 247.16it/s]


Starting evaluation
ROOT:  /rds/user/fz287/hpc-work/MLMI4/results\gp\lbanp\lbanp-num_latents-64_rbf


100%|##########| 3000/3000 [01:37<00:00, 30.86it/s]
lbanp:lbanp-num_latents-64_rbf rbf tar_ll 1.2739 loss -1.2739 (97.228 secs)


Eval has finished
Overriding argument num_latents: 64
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
generating evaluation sets...
Generating Evaluation Sets with rbf kernel
Generating Evaluation sets with context and target points:   None None


...

True
generating evaluation sets...
Generating Evaluation Sets with rbf kernel
Generating Evaluation sets with context and target points:   None None


100%|##########| 3000/3000 [00:10<00:00, 274.36it/s]


Starting evaluation
ROOT:  /rds/user/fz287/hpc-work/MLMI4/results\gp\lbanp\lbanp-num_latents-64_rbf


100%|##########| 3000/3000 [01:41<00:00, 29.55it/s]
lbanp:lbanp-num_latents-64_rbf rbf tar_ll 1.2678 loss -1.2678 (101.523 secs)


Eval has finished


In [23]:
model_names=['lbanp-num_latents-8',
             'lbanp-num_latents-128_rbf']
latents_l=[8, 128]
modelito='lbanp'

for latents, model_name in zip(latents_l, model_names):
    print('###############MODEL: ',model_name, latents)
    for seed in [0,58,100,250,550,1200]:
        print('###############SEED: ',seed)
        %run gp_track.py --mode eval --expid {model_name} --model {modelito} --num_latents {latents} --custom_eval True --eval_num_batches 3000 --eval_seed {seed}

###############MODEL:  lbanp-num_latents-8 8
###############SEED:  0
Overriding argument num_latents: 8
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
True
generating evaluation sets...
Generating Evaluation Sets with rbf kernel
Generating Evaluation sets with context and target points:   None None


100%|##########| 3000/3000 [00:10<00:00, 278.61it/s]


Starting evaluation
ROOT:  /rds/user/fz287/hpc-work/MLMI4/results\gp\lbanp\lbanp-num_latents-8


100%|##########| 3000/3000 [01:51<00:00, 26.95it/s]
lbanp:lbanp-num_latents-8 rbf tar_ll -0.5588 loss 0.5588 (111.325 secs)


Eval has finished
###############SEED:  58
Overriding argument num_latents: 8
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
True
generating evaluation sets...
Generating Evaluation Sets with rbf kernel
Generating Evaluation sets with context and target points:   None None


...

Eval has finished
###############SEED:  1200
Overriding argument num_latents: 128
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
True
generating evaluation sets...
Generating Evaluation Sets with rbf kernel
Generating Evaluation sets with context and target points:   None None


100%|##########| 3000/3000 [00:11<00:00, 270.05it/s]


Starting evaluation
ROOT:  /rds/user/fz287/hpc-work/MLMI4/results\gp\lbanp\lbanp-num_latents-128_rbf


100%|##########| 3000/3000 [02:07<00:00, 23.62it/s]
lbanp:lbanp-num_latents-128_rbf rbf tar_ll 1.0749 loss -1.0749 (127.029 secs)


Eval has finished


In [24]:
model_names=['isanp-num_latents-8_rbf',
             'isanp-num_latents-16_rbf_0',
             'isanp-num_latents-32_rbf_0',
             'isanp-num_latents-64_rbf_0',
             'isanp-num_latents-128_rbf_0',
            'isanp-num_latents-256_rbf_0'
            ]
latents_l=[8, 16, 32,64, 128,256]
modelito='isanp'

for latents, model_name in zip(latents_l, model_names):
    print('###############MODEL: ',model_name, latents)
    for seed in [0,58,100,250,550,1200]:
        print('###############SEED: ',seed)
        %run gp_track.py --mode eval --expid {model_name} --model {modelito} --num_latents {latents} --custom_eval True --eval_num_batches 3000 --eval_seed {seed}

###############MODEL:  isanp-num_latents-8_rbf 8
###############SEED:  0
Overriding argument num_latents: 8
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
True
generating evaluation sets...
Generating Evaluation Sets with rbf kernel
Generating Evaluation sets with context and target points:   None None


100%|##########| 3000/3000 [00:12<00:00, 232.27it/s]


Starting evaluation
ROOT:  /rds/user/fz287/hpc-work/MLMI4/results\gp\isanp\isanp-num_latents-8_rbf


100%|##########| 3000/3000 [02:24<00:00, 20.77it/s]
isanp:isanp-num_latents-8_rbf rbf tar_ll 0.1894 loss -0.1894 (144.464 secs)


Eval has finished
###############SEED:  58
Overriding argument num_latents: 8
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
True
generating evaluation sets...
Generating Evaluation Sets with rbf kernel
Generating Evaluation sets with context and target points:   None None


...

Eval has finished
###############SEED:  1200
Overriding argument num_latents: 256
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
True
generating evaluation sets...
Generating Evaluation Sets with rbf kernel
Generating Evaluation sets with context and target points:   None None


100%|##########| 3000/3000 [00:12<00:00, 248.84it/s]


Starting evaluation
ROOT:  /rds/user/fz287/hpc-work/MLMI4/results\gp\isanp\isanp-num_latents-256_rbf_0


100%|##########| 3000/3000 [01:40<00:00, 29.85it/s]
isanp:isanp-num_latents-256_rbf_0 rbf tar_ll 1.3468 loss -1.3468 (100.512 secs)


Eval has finished


In [25]:
model_names=['isanp2-num_latents-8_rbf_100',
             'isanp2-num_latents-16_rbf_100',
             'isanp2-num_latents-32_rbf_100',
             'isanp2-num_latents-64_rbf_100',
             'isanp2-num_latents-128_rbf_100',
            'isanp2-num_latents-256_rbf_100']
latents_l=[8, 16, 32,64, 128,256]
modelito='isanp2'

for latents, model_name in zip(latents_l, model_names):
    print('###############MODEL: ',model_name, latents)
    for seed in [0,58,100,250,550,1200]:
        print('###############SEED: ',seed)
        %run gp_track.py --mode eval --expid {model_name} --model {modelito} --num_latents {latents} --custom_eval True --eval_num_batches 3000 --eval_seed {seed}

###############MODEL:  isanp2-num_latents-8_rbf_100 8
###############SEED:  0
Overriding argument num_latents: 8
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
True
generating evaluation sets...
Generating Evaluation Sets with rbf kernel
Generating Evaluation sets with context and target points:   None None


100%|##########| 3000/3000 [00:08<00:00, 369.67it/s]


Starting evaluation
ROOT:  /rds/user/fz287/hpc-work/MLMI4/results\gp\isanp2\isanp2-num_latents-8_rbf_100


100%|##########| 3000/3000 [01:31<00:00, 32.68it/s]
isanp2:isanp2-num_latents-8_rbf_100 rbf tar_ll 1.2115 loss -1.2115 (91.800 secs)


Eval has finished
###############SEED:  58
Overriding argument num_latents: 8
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
True
generating evaluation sets...
Generating Evaluation Sets with rbf kernel
Generating Evaluation sets with context and target points:   None None


...

Eval has finished
###############SEED:  1200
Overriding argument num_latents: 256
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
True
generating evaluation sets...
Generating Evaluation Sets with rbf kernel
Generating Evaluation sets with context and target points:   None None


100%|##########| 3000/3000 [00:10<00:00, 284.73it/s]


Starting evaluation
ROOT:  /rds/user/fz287/hpc-work/MLMI4/results\gp\isanp2\isanp2-num_latents-256_rbf_100


100%|##########| 3000/3000 [02:01<00:00, 24.68it/s]
isanp2:isanp2-num_latents-256_rbf_100 rbf tar_ll 1.1505 loss -1.1505 (121.567 secs)


Eval has finished


In [ ]:
model_names=['lbanp-num_latents-8_rbf',
             'lbanp-num_latents-16_rbf',
             'lbanp-num_latents-32_rbf',
             'lbanp-num_latents-128_rbf',
            'lbanp-num_latents-256_rbf']
latents_l=[8, 16, 32, 128,256]
modelito='lbanp'

for latents, model_name in zip(latents_l, model_names):
    print('###############MODEL: ',model_name, latents)
    for seed in [0,58,100,250,550,1200]:
        print('###############SEED: ',seed)
        %run gp_track.py --mode eval --expid {model_name} --model {modelito} --num_latents {latents} --custom_eval True --eval_num_batches 3000 --eval_seed {seed}

In [ ]:
for seed in [0,58,100,250,550,1200]:
    print('###############SEED: ',seed)
    %run gp_track.py --mode eval --expid np_rbf --model np --custom_eval True --eval_num_batches 3000 --eval_seed {seed}

In [ ]:
model_names=['isanp-num_latents-8_rbf']
latents_l=[8, 16, 32,64, 128,256]
modelito='isanp'

for latents, model_name in zip(latents_l, model_names):
    print('###############MODEL: ',model_name, latents)
    for seed in [0,58,100,250,550,1200]:
        print('###############SEED: ',seed)
        %run gp_track.py --mode eval --expid {model_name} --model {modelito} --num_latents {latents} --custom_eval True --eval_num_batches 3000 --eval_seed {seed}

In [1]:
for seed in [0,58,100,250,550,1200]:
    print('###############SEED: ',seed)
    %run gp_track.py --mode eval --expid cnp --model cnp --custom_eval True --eval_num_batches 3000 --eval_seed {seed}

###############SEED:  0
True
generating evaluation sets...
Generating Evaluation Sets with rbf kernel
Generating Evaluation sets with context and target points:   None None


100%|##########| 3000/3000 [00:04<00:00, 733.61it/s]


Starting evaluation
ROOT:  /rds/user/fz287/hpc-work/MLMI4/results\gp\cnp\cnp


100%|##########| 3000/3000 [00:04<00:00, 640.48it/s]
cnp:cnp rbf ctx_ll 0.7169 tar_ll 0.2602 (4.686 secs)


Eval has finished
###############SEED:  58
True
generating evaluation sets...
Generating Evaluation Sets with rbf kernel
Generating Evaluation sets with context and target points:   None None


...

Eval has finished
###############SEED:  1200
True
generating evaluation sets...
Generating Evaluation Sets with rbf kernel
Generating Evaluation sets with context and target points:   None None


100%|##########| 3000/3000 [00:05<00:00, 585.71it/s]


Starting evaluation
ROOT:  /rds/user/fz287/hpc-work/MLMI4/results\gp\cnp\cnp


100%|##########| 3000/3000 [00:05<00:00, 562.72it/s]
cnp:cnp rbf ctx_ll 0.7093 tar_ll 0.2486 (5.332 secs)


Eval has finished


In [2]:
model_names=['lbanp-num_latents-128_rbf_0']
latents_l=[128]
modelito='lbanp'

for latents, model_name in zip(latents_l, model_names):
    print('###############MODEL: ',model_name, latents)
    for seed in [0,58,100,250,550,1200]:
        print('###############SEED: ',seed)
        %run gp_track.py --mode eval --expid {model_name} --model {modelito} --num_latents {latents} --custom_eval True --eval_num_batches 3000 --eval_seed {seed}

###############MODEL:  lbanp-num_latents-128_rbf_0 128
###############SEED:  0
Overriding argument num_latents: 128
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
True
generating evaluation sets...
Generating Evaluation Sets with rbf kernel
Generating Evaluation sets with context and target points:   None None


100%|##########| 3000/3000 [00:04<00:00, 605.37it/s]


Starting evaluation
ROOT:  /rds/user/fz287/hpc-work/MLMI4/results\gp\lbanp\lbanp-num_latents-128_rbf_0


100%|##########| 3000/3000 [00:43<00:00, 69.47it/s]
lbanp:lbanp-num_latents-128_rbf_0 rbf tar_ll 1.2540 loss -1.2540 (43.185 secs)


Eval has finished
###############SEED:  58
Overriding argument num_latents: 128
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
True
generating evaluation sets...
Generating Evaluation Sets with rbf kernel
Generating Evaluation sets with context and target points:   None None


...

Eval has finished
###############SEED:  1200
Overriding argument num_latents: 128
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
True
generating evaluation sets...
Generating Evaluation Sets with rbf kernel
Generating Evaluation sets with context and target points:   None None


100%|##########| 3000/3000 [00:04<00:00, 712.68it/s]


Starting evaluation
ROOT:  /rds/user/fz287/hpc-work/MLMI4/results\gp\lbanp\lbanp-num_latents-128_rbf_0


100%|##########| 3000/3000 [00:37<00:00, 79.91it/s]
lbanp:lbanp-num_latents-128_rbf_0 rbf tar_ll 1.2518 loss -1.2518 (37.540 secs)


Eval has finished


In [10]:
model_names=['lbanp-num_latents-256_rbf_']
latents_l=[256]
modelito='lbanp'

for latents, model_name in zip(latents_l, model_names):
    print('###############MODEL: ',model_name, latents)
    for seed in [0,58,100,250,550,1200]:
        print('###############SEED: ',seed)
        %run gp_track.py --mode eval --expid {model_name} --model {modelito} --num_latents {latents} --custom_eval True --eval_num_batches 3000 --eval_seed {seed}

###############MODEL:  lbanp-num_latents-256_rbf_ 256
###############SEED:  0
Overriding argument num_latents: 256
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
True
generating evaluation sets...
Generating Evaluation Sets with rbf kernel
Generating Evaluation sets with context and target points:   None None


100%|##########| 3000/3000 [00:03<00:00, 758.46it/s]


Starting evaluation
ROOT:  /rds/user/fz287/hpc-work/MLMI4/results\gp\lbanp\lbanp-num_latents-256_rbf_


100%|##########| 3000/3000 [00:48<00:00, 62.48it/s]
lbanp:lbanp-num_latents-256_rbf_ rbf tar_ll 0.8236 loss -0.8236 (48.016 secs)


Eval has finished
###############SEED:  58
Overriding argument num_latents: 256
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
True
generating evaluation sets...
Generating Evaluation Sets with rbf kernel
Generating Evaluation sets with context and target points:   None None


...

Eval has finished
###############SEED:  1200
Overriding argument num_latents: 256
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
True
generating evaluation sets...
Generating Evaluation Sets with rbf kernel
Generating Evaluation sets with context and target points:   None None


100%|##########| 3000/3000 [00:04<00:00, 603.27it/s]


Starting evaluation
ROOT:  /rds/user/fz287/hpc-work/MLMI4/results\gp\lbanp\lbanp-num_latents-256_rbf_


100%|##########| 3000/3000 [00:43<00:00, 68.19it/s]
lbanp:lbanp-num_latents-256_rbf_ rbf tar_ll 0.8130 loss -0.8130 (43.995 secs)


Eval has finished


### Mattern

In [3]:
model_names=['cnp_matern','canp_matern','np_matern','anp_matern','bnp_matern','tnpd_matern','tnpa_matern']
modelitos=['cnp','canp','np','anp','bnp','tnpd','tnpa']

for model_name, modelito in zip(model_names, modelitos):
    print('###############MODEL: ',model_name)
    for seed in [0,58,100,250,550,1200]:
        print('###############SEED: ',seed)
        %run gp_track.py --mode eval --expid {model_name} --model {modelito} --custom_eval True --eval_num_batches 3000 --eval_seed {seed} --eval_kernel matern

###############MODEL:  cnp_matern
###############SEED:  0
True
generating evaluation sets...
Generating Evaluation Sets with matern kernel
Generating Evaluation sets with context and target points:   None None


100%|##########| 3000/3000 [00:04<00:00, 657.35it/s]


Starting evaluation
ROOT:  /rds/user/fz287/hpc-work/MLMI4/results\gp\cnp\cnp_matern


100%|##########| 3000/3000 [00:04<00:00, 702.33it/s]
cnp:cnp_matern matern ctx_ll 0.5641 tar_ll 0.0308 (4.273 secs)


Eval has finished
###############SEED:  58
generating evaluation sets...
Generating Evaluation Sets with matern kernel
Generating Evaluation sets with context and target points:   None None


...

Eval has finished
###############SEED:  1200
Overriding argument pretrain: False
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
True
generating evaluation sets...
Generating Evaluation Sets with matern kernel
Generating Evaluation sets with context and target points:   None None


100%|##########| 3000/3000 [00:09<00:00, 330.75it/s]


Starting evaluation
ROOT:  /rds/user/fz287/hpc-work/MLMI4/results\gp\tnpa\tnpa_matern


100%|##########| 3000/3000 [00:22<00:00, 132.04it/s]
tnpa:tnpa_matern matern tar_ll 1.2147 loss -1.2147 (22.724 secs)


Eval has finished


In [4]:
model_names=['lbanp-num_latents-8_matern',
             'lbanp-num_latents-128_matern'
            ]
latents_l=[8, 128]
modelito='lbanp'

for latents, model_name in zip(latents_l, model_names):
    print('###############MODEL: ',model_name, latents)
    for seed in [0,58,100,250,550,1200]:
        print('###############SEED: ',seed)
        %run gp_track.py --mode eval --expid {model_name} --model {modelito} --num_latents {latents} --custom_eval True --eval_num_batches 3000 --eval_seed {seed} --eval_kernel matern

###############MODEL:  lbanp-num_latents-8_matern 8
###############SEED:  0
Overriding argument num_latents: 8
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
True
generating evaluation sets...
Generating Evaluation Sets with matern kernel
Generating Evaluation sets with context and target points:   None None


100%|##########| 3000/3000 [00:08<00:00, 357.09it/s]


Starting evaluation
ROOT:  /rds/user/fz287/hpc-work/MLMI4/results\gp\lbanp\lbanp-num_latents-8_matern


100%|##########| 3000/3000 [01:06<00:00, 45.07it/s]
lbanp:lbanp-num_latents-8_matern matern tar_ll 0.7765 loss -0.7765 (66.562 secs)


Eval has finished
###############SEED:  58
Overriding argument num_latents: 8
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
True
generating evaluation sets...
Generating Evaluation Sets with matern kernel
Generating Evaluation sets with context and target points:   None None


...

Eval has finished
###############SEED:  1200
Overriding argument num_latents: 128
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
True
generating evaluation sets...
Generating Evaluation Sets with matern kernel
Generating Evaluation sets with context and target points:   None None


100%|##########| 3000/3000 [00:08<00:00, 335.61it/s]


Starting evaluation
ROOT:  /rds/user/fz287/hpc-work/MLMI4/results\gp\lbanp\lbanp-num_latents-128_matern


100%|##########| 3000/3000 [00:56<00:00, 53.31it/s]
lbanp:lbanp-num_latents-128_matern matern tar_ll 0.8898 loss -0.8898 (56.278 secs)


Eval has finished


In [5]:
model_names=['isanp-num_latents-8_matern',
             'isanp-num_latents-128_matern',
            ]
latents_l=[8, 128]
modelito='isanp'

for latents, model_name in zip(latents_l, model_names):
    print('###############MODEL: ',model_name, latents)
    for seed in [0,58,100,250,550,1200]:
        print('###############SEED: ',seed)
        %run gp_track.py --mode eval --expid {model_name} --model {modelito} --num_latents {latents} --custom_eval True --eval_num_batches 3000 --eval_seed {seed} --eval_kernel matern

###############MODEL:  isanp-num_latents-8_matern 8
###############SEED:  0
Overriding argument num_latents: 8
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
True
generating evaluation sets...
Generating Evaluation Sets with matern kernel
Generating Evaluation sets with context and target points:   None None


100%|##########| 3000/3000 [00:08<00:00, 335.14it/s]


Starting evaluation
ROOT:  /rds/user/fz287/hpc-work/MLMI4/results\gp\isanp\isanp-num_latents-8_matern


100%|##########| 3000/3000 [01:09<00:00, 43.46it/s]
isanp:isanp-num_latents-8_matern matern tar_ll 0.8181 loss -0.8181 (69.029 secs)


Eval has finished
###############SEED:  58
Overriding argument num_latents: 8
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
True
generating evaluation sets...
Generating Evaluation Sets with matern kernel
Generating Evaluation sets with context and target points:   None None


...

Eval has finished
###############SEED:  1200
Overriding argument num_latents: 128
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
True
generating evaluation sets...
Generating Evaluation Sets with matern kernel
Generating Evaluation sets with context and target points:   None None


100%|##########| 3000/3000 [00:05<00:00, 580.79it/s]


Starting evaluation
ROOT:  /rds/user/fz287/hpc-work/MLMI4/results\gp\isanp\isanp-num_latents-128_matern


100%|##########| 3000/3000 [00:46<00:00, 63.96it/s]
isanp:isanp-num_latents-128_matern matern tar_ll 0.9133 loss -0.9133 (46.904 secs)


Eval has finished


In [8]:
model_names=['isanp2-num_latents-8_matern_0',
             'isanp2-num_latents-128_matern_0',
            ]
latents_l=[8, 128]
modelito='isanp2'

for latents, model_name in zip(latents_l, model_names):
    print('###############MODEL: ',model_name, latents)
    for seed in [0,58,100,250,550,1200]:
        print('###############SEED: ',seed)
        %run gp_track.py --mode eval --expid {model_name} --model {modelito} --num_latents {latents} --custom_eval True --eval_num_batches 3000 --eval_seed {seed} --eval_kernel matern

###############MODEL:  isanp2-num_latents-8_matern_0 8
###############SEED:  0
Overriding argument num_latents: 8
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
True
generating evaluation sets...
Generating Evaluation Sets with matern kernel
Generating Evaluation sets with context and target points:   None None


100%|##########| 3000/3000 [00:04<00:00, 720.85it/s]


Starting evaluation
ROOT:  /rds/user/fz287/hpc-work/MLMI4/results\gp\isanp2\isanp2-num_latents-8_matern_0


100%|##########| 3000/3000 [00:47<00:00, 63.51it/s]
isanp2:isanp2-num_latents-8_matern_0 matern tar_ll 0.7646 loss -0.7646 (47.241 secs)


Eval has finished
###############SEED:  58
Overriding argument num_latents: 8
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
True
generating evaluation sets...
Generating Evaluation Sets with matern kernel
Generating Evaluation sets with context and target points:   None None


...

Eval has finished
###############SEED:  1200
Overriding argument num_latents: 128
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
True
generating evaluation sets...
Generating Evaluation Sets with matern kernel
Generating Evaluation sets with context and target points:   None None


100%|##########| 3000/3000 [00:05<00:00, 573.15it/s]


Starting evaluation
ROOT:  /rds/user/fz287/hpc-work/MLMI4/results\gp\isanp2\isanp2-num_latents-128_matern_0


100%|##########| 3000/3000 [00:43<00:00, 68.50it/s]
isanp2:isanp2-num_latents-128_matern_0 matern tar_ll 0.8258 loss -0.8258 (43.800 secs)


Eval has finished


In [ ]:
for seed in [0,58,100,250,550,1200]:
    print('###############SEED: ',seed)
    %run gp_track.py --mode eval --expid tnpd --model tnpd --custom_eval True --eval_num_batches 3000 --eval_seed {seed}

###############SEED:  0
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
True
generating evaluation sets...
Generating Evaluation Sets with rbf kernel
Generating Evaluation sets with context and target points:   None None


100%|##########| 3000/3000 [00:07<00:00, 427.07it/s]


Starting evaluation
ROOT:  /rds/user/fz287/hpc-work/MLMI4/results\gp\tnpd\tnpd


100%|##########| 3000/3000 [00:17<00:00, 171.97it/s]
tnpd:tnpd rbf tar_ll 1.0365 loss -1.0365 (17.447 secs)


Eval has finished
###############SEED:  58
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
True
generating evaluation sets...
Generating Evaluation Sets with rbf kernel
Generating Evaluation sets with context and target points:   None None


100%|##########| 3000/3000 [00:07<00:00, 375.01it/s]


Starting evaluation
ROOT:  /rds/user/fz287/hpc-work/MLMI4/results\gp\tnpd\tnpd


100%|##########| 3000/3000 [00:21<00:00, 140.28it/s]
tnpd:tnpd rbf tar_ll 1.0245 loss -1.0245 (21.388 secs)


Eval has finished
###############SEED:  100
Overriding argument d_model: 64
Overriding argument emb_depth: 4
Overriding argument dim_feedforward: 128
Overriding argument nhead: 4
Overriding argument dropout: 0.0
Overriding argument num_layers: 6
True
generating evaluation sets...
Generating Evaluation Sets with rbf kernel
Generating Evaluation sets with context and target points:   None None


 25%|##5       | 756/3000 [00:01<00:05, 419.41it/s]